In [1]:
import numpy as np
import math

In [2]:
# Load DataSet
X_seen=np.load('X_seen.npy',encoding='bytes',allow_pickle=True) 	# (40 x N_i x D): 40 feature matrices. X_seen[i] is the N_i x D feature matrix of seen class i

Xtest=np.load('Xtest.npy')	# (6180, 4096): feature matrix of the test data.

Ytest=np.load('Ytest.npy',)	 # (6180, 1): ground truth labels of the test data

class_attributes_seen=np.load('class_attributes_seen.npy')	# (40, 85): 40x85 matrix with each row being the 85-dimensional class attribute vector of a seen class.

class_attributes_unseen=np.load('class_attributes_unseen.npy')	# (10, 85): 10x85 matrix with each row being the 85-dimensional class attribute vector of an  unseen class.

In [3]:
# Calculate Mean of each class
X_class_mean = []

for i in range(40):
    class_mean = np.mean(X_seen[i], axis=0)
    X_class_mean.append(class_mean)


In [4]:
# Compute the similarity (dot product based) of each unseen class with each of the seen classes.
similarity = []   # 10 *40

for i in range(10):
    similarity_i = []
    for j in range(40):
        similarity_i.append(np.dot(class_attributes_unseen[i],class_attributes_seen[j]))
    similarity.append(similarity_i)

In [5]:
# Normalize the similarity vector (to that it sums to 1, since we are using a convex combination).
normalized_similarity_vector = []
for i in range(10):
    sum = 0
    normalized_similarity_vector_i = []
    for j in range(40):
        sum += similarity[i][j]
    for j in range(40):
        normalized_similarity_vector_i.append(similarity[i][j]/sum)
    normalized_similarity_vector.append(normalized_similarity_vector_i)

In [6]:
# Compute the mean of each unseen class using a convex combination of means of seen classes.
mean_unseen_class = []
for i in range(10):
    normalized_similarity_vector_numpy = np.array(normalized_similarity_vector[i]).reshape(1, -1)
    X_class_mean_numpy = np.array(X_class_mean)
    mean = np.dot(normalized_similarity_vector_numpy,X_class_mean_numpy)
    mean_unseen_class.append(mean.reshape(-1))

In [7]:
# Apply the model to predict labels on unseen class test inputs. Xtest

predicted_labels = []
for i in range(len(Xtest)):
    distances = []
    for j in range(10):
        distance = np.linalg.norm(Xtest[i]-mean_unseen_class[j])
        distances.append(distance)
    predict_value=np.argmin(distances)+1
    predicted_labels.append(predict_value)

In [8]:
# Compute classification accuracies.
correct_prediction = 0
total_prediction = len(Ytest)
for i in range(total_prediction):
    if(predicted_labels[i]==Ytest[i]):
        correct_prediction+=1

accuracy = correct_prediction/total_prediction*100
print("Accuracy : {}".format(accuracy))

Accuracy : 46.89320388349515
